In [1]:
import pandas as pd 
import sqlite3 
from sqlite3 import Error 
import json

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def create_place(conn, place):
    sql = """
        INSERT INTO places(airport, placeId, city, state)
        VALUES(?,?,?,?)
    """
    cur = conn.cursor()
    cur.execute(sql, place)
    return cur.lastrowid

conn = create_connection("./flights.db")

sql_create_places_table = """CREATE TABLE IF NOT EXISTS places (
                                airport text PRIMARY KEY, 
                                placeId text NOT NULL, 
                                city text, 
                                state text
                                );
                            """

if conn is not None: 
    create_table(conn, sql_create_places_table)
    
    


2.6.0


In [7]:
# populate places table with toy data

more = [('ATL', 'ATL-sky', 'Atlanta', 'GA'),
       ('LAX', 'LAX-sky', 'Los Angeles', 'CA'),
       ('ORD', 'ORD-sky', 'Chicago', 'IL'),
       ('DFW', 'DFW-sky', 'Dallas/Fort Worth', 'TX'),
       ('DEN', 'DEN-sky', 'Denver', 'CO'),
       ('SFO', 'SFO-sky', 'San Francisco', 'CA'),
       ('SEA', 'SEA-sky', 'Seattle/Tacoma', 'WA'),
       ('JFK', 'JFK-sky', 'New York City', 'NY'),
       ('RDU', 'RDU-sky', 'Raleigh/Durham', 'NC'),
       ('LAS', 'LAS-sky', 'Las Vegas', 'NV'),
        ('MCO', 'MCO-sky', 'Orlando', 'FL'),
        ('EWR', 'EWR-sky', 'New York/Newark', 'NJ'),
        ('CLT', 'CLT-sky', 'Charlotte', 'NC'),
        ('PHX', 'PHX-sky', 'Pheonix', 'AZ'),
        ('IAH', 'IAH-sky', 'Houston', 'TX'),
        ('MIA', 'MIA-sky', 'Miami', 'FL'),
        ('BOS', 'BOS-sky', 'Boston', 'MA'),
        ('MSP', 'MSP-sky', 'Minneapolis/St. Paul', 'MN'),
        ('FLL', 'FLL-sky', 'Fort Lauderdale', 'FL'),
        ('DTW', 'DTW-sky', 'Detroit', 'MI'),
        ('PHL', 'PHL-sky', 'Philadelphia', 'PA'),
        ('LGA', 'LGA-sky', 'New York', 'NY'),
        ('BWI', 'BWI-sky', 'Baltimore', 'MD'),
        ('SLC', 'SLC-sky', 'Salt Lake City', 'UT'),
        ('SAN', 'SAN-sky', 'San Diego', 'CA'),
        ('IAD', 'IAD-sky', 'Washington, D.C.', 'VA'),
        ('DCA', 'DCA-sky', 'Washington, D.C.', 'VA'),
        ('MDW', 'MDW-sky', 'Chicago', 'IL'),
        ('TPA', 'TPA-sky', 'Tampa', 'FL'),
        ('PDX', 'PDX-sky', 'Portland', 'OR'),
        ('HNL', 'HNL-sky', 'Honolulu', 'HI'),
       ]

for i in more:
    try:
        create_place(conn, i)
    except Error as e:
        print(e)


database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked
database is locked


In [3]:
# check if table is populated

cur = conn.cursor() 
cur.execute("SELECT * from places")
rows = cur.fetchall()
for row in rows: 
    print(row)

In [4]:
# see table using pandas 
df = pd.read_sql_query("select * from places", conn)
df

,airport,placeId,city,state


In [5]:
# flight api - create session 
import requests 


def create_session(originPlace, destinationPlace, date):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/pricing/v1.0"
    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "0dc1882529mshd78b1b907f784dap1b1d60jsn3530abf3f164",
    'content-type': "application/x-www-form-urlencoded"
    }
    payload = "country=US&currency=USD&locale=en-US&originPlace={}&destinationPlace={}&outboundDate={}&adults=1".format(originPlace, destinationPlace, date)
    response = requests.request("POST", url, data=payload, headers=headers)
    if 'Location' in response.headers:
        return response.headers["Location"].split("/")[-1]
    else: 
        return None
    
def poll_results(originPlace, destinationPlace, date):
    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "0dc1882529mshd78b1b907f784dap1b1d60jsn3530abf3f164"
    }
    querystring = {"pageIndex":"0","pageSize":"10"}
    session_key = create_session(originPlace, destinationPlace, date)
    if session_key is not None: 
        url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/pricing/uk2/v1.0/{}".format(session_key)
        response = requests.request("GET", url, headers=headers, params=querystring)
        return response
    else: 
        return None

# res = poll_results("JFK-sky", "DFW-sky", "2019-11-01").text
# json_res = json.loads(res)



In [6]:
# flight api - poll results for all combinations 

pairs = []

# top 10 airports 
for i in more[:10]: 
    for j in more[:10]: 
        if i[1] != j[1]:
            pairs.append((i[1], j[1]))

big_res = []
for pair in pairs: 
    res = poll_results(pair[0], pair[1], "2019-11-10").text
    big_res.append({"path": (pair[0], pair[1]), "result": json.loads(res)})

print('Querying complete')
    

hello


In [25]:
# parse result 
import pprint 
def parse_segments(segment_ids, segments, places, carriers):
    all_segments_info = []
    for seg_id in segment_ids:
        segment_info = {}
        segment_info["carrier"] = None
        segment_info["duration"] = None
        segment_info["from_place"] = None
        segment_info["to_place"] = None
        for segment in segments:
            if segment["Id"] == seg_id:
                
                # input carrier information
                carrier_id = segment["Carrier"]
                for c in carriers: 
                    if c["Id"] == carrier_id:
                        segment_info["carrier"] = {"Code": c["Code"], "Name": c["Name"]}
                
                segment_info["duration"] = segment["Duration"]
                
                # input place information 
                for place in places: 
                    if "Code" not in place:
                        continue
                        
                    if place["Id"] == segment["OriginStation"]:
                        segment_info["from_place"] = {"Code": place["Code"], "Name": place["Name"]}
                    if place["Id"] == segment["DestinationStation"]:
                        segment_info["to_place"] = {"Code": place["Code"], "Name": place["Name"]}                
        
        all_segments_info.append(segment_info)
    return all_segments_info
                
def parse_results(result):
    all_flights_info = []
    flight_options = result["Itineraries"]
    legs = result["Legs"]
    segments = result["Segments"]
    places = result["Places"]
    carriers = result["Carriers"]
    for flight in flight_options:
        flight_info = {}
        flight_info["duration"] = None
        flight_info["departure_time"] = None
        flight_info["arrival_time"] = None
        flight_info["segments"] = None
        flight_info["price"] = None
        
        leg_id = flight["OutboundLegId"]
        segment_ids = None
        
        # get duration, departure and arrival time 
        for leg in legs: 
            if leg["Id"] == leg_id:
                flight_info["duration"] = leg["Duration"]
                flight_info["departure_time"] = leg["Departure"]
                flight_info["arrival_time"] = leg["Arrival"]
                
                segment_ids = leg["SegmentIds"]
                flight_info["segments"] = parse_segments(segment_ids, segments, places, carriers)
        
        
        # pricing info 
        pricing_options = flight["PricingOptions"]
        min_cost = float("inf")
        for pricing_option in pricing_options:
            if pricing_option["Price"] < min_cost:
                min_cost = pricing_option["Price"]
        flight_info["price"] = min_cost
                
        
        all_flights_info.append(flight_info)
    return all_flights_info
                

# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(parse_results(json_res))

[]


In [26]:
# Results 

result = []
for res in big_res:
    result.append({"path": res["path"], "result": parse_results(res["result"])})

for res in result: 
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(res)

{   'path': ('ATL-sky', 'LAX-sky'),
    'result': [   {   'arrival_time': '2019-11-10T11:50:00',
                      'departure_time': '2019-11-10T07:10:00',
                      'duration': 460,
                      'price': 287.01,
                      'segments': [   {   'carrier': {   'Code': 'WN',
                                                         'Name': 'Southwest '
                                                                 'Airlines'},
                                          'duration': 250,
                                          'from_place': {   'Code': 'ATL',
                                                            'Name': 'Atlanta '
                                                                    'Hartsfield-Jackson'},
                                          'to_place': {   'Code': 'PHX',
                                                          'Name': 'Phoenix Sky '
                                                                  'Harbor'}},

                                                         'Name': 'United'},
                                          'duration': 135,
                                          'from_place': {   'Code': 'ATL',
                                                            'Name': 'Atlanta '
                                                                    'Hartsfield-Jackson'},
                                          'to_place': {   'Code': 'EWR',
                                                          'Name': 'New York '
                                                                  'Newark'}},
                                      {   'carrier': {   'Code': 'UA',
                                                         'Name': 'United'},
                                          'duration': 240,
                                          'from_place': {   'Code': 'EWR',
                                                            'Name': 'New York '
                              

                                                            'Name': 'Atlanta '
                                                                    'Hartsfield-Jackson'},
                                          'to_place': {   'Code': 'MCI',
                                                          'Name': 'Kansas City '
                                                                  'International'}},
                                      {   'carrier': {   'Code': 'WN',
                                                         'Name': 'Southwest '
                                                                 'Airlines'},
                                          'duration': 190,
                                          'from_place': {   'Code': 'MCI',
                                                            'Name': 'Kansas '
                                                                    'City '
                                                                    'Interna

                                                         'Name': 'jetBlue'},
                                          'duration': 165,
                                          'from_place': {   'Code': 'FLL',
                                                            'Name': 'Fort '
                                                                    'Lauderdale '
                                                                    'International'},
                                          'to_place': {   'Code': 'JFK',
                                                          'Name': 'New York '
                                                                  'John F. '
                                                                  'Kennedy'}}]},
                  {   'arrival_time': '2019-11-10T21:15:00',
                      'departure_time': '2019-11-10T16:05:00',
                      'duration': 310,
                      'price': 479.0,
                      'segments': [

                                                            'Name': 'Seattle / '
                                                                    'Tacoma '
                                                                    'International'},
                                          'to_place': {   'Code': 'PDX',
                                                          'Name': 'Portland'}},
                                      {   'carrier': {   'Code': 'AS',
                                                         'Name': 'Alaska '
                                                                 'Airlines'},
                                          'duration': 124,
                                          'from_place': {   'Code': 'PDX',
                                                            'Name': 'Portland'},
                                          'to_place': {   'Code': 'LAS',
                                                          'Name': 'Las Vegas '
           

...

                      'segments': [   {   'carrier': {   'Code': 'B6',
                                                         'Name': 'jetBlue'},
                                          'duration': 293,
                                          'from_place': {   'Code': 'LAS',
                                                            'Name': 'Las Vegas '
                                                                    'Mccarran'},
                                          'to_place': {   'Code': 'BOS',
                                                          'Name': 'Boston '
                                                                  'Logan '
                                                                  'International'}},
                                      {   'carrier': {   'Code': 'B6',
                                                         'Name': 'jetBlue'},
                                          'duration': 199,
                                        

                      'segments': [   {   'carrier': {   'Code': 'UA',
                                                         'Name': 'United'},
                                          'duration': 254,
                                          'from_place': {   'Code': 'LAS',
                                                            'Name': 'Las Vegas '
                                                                    'Mccarran'},
                                          'to_place': {   'Code': 'IAD',
                                                          'Name': 'Washington '
                                                                  'Dulles'}},
                                      {   'carrier': {   'Code': 'UA',
                                                         'Name': 'United'},
                                          'duration': 216,
                                          'from_place': {   'Code': 'IAD',
                                             

                                          'from_place': {   'Code': 'LAX',
                                                            'Name': 'Los '
                                                                    'Angeles '
                                                                    'International'},
                                          'to_place': {   'Code': 'SFO',
                                                          'Name': 'San '
                                                                  'Francisco '
                                                                  'International'}}]},
                  {   'arrival_time': '2019-11-10T09:05:00',
                      'departure_time': '2019-11-10T07:25:00',
                      'duration': 100,
                      'price': 141.99,
                      'segments': [   {   'carrier': {   'Code': 'WN',
                                                         'Name': 'Southwest '
                      

                      'duration': 481,
                      'price': 476.0,
                      'segments': [   {   'carrier': {   'Code': 'B6',
                                                         'Name': 'jetBlue'},
                                          'duration': 302,
                                          'from_place': {   'Code': 'LAS',
                                                            'Name': 'Las Vegas '
                                                                    'Mccarran'},
                                          'to_place': {   'Code': 'BOS',
                                                          'Name': 'Boston '
                                                                  'Logan '
                                                                  'International'}},
                                      {   'carrier': {   'Code': 'B6',
                                                         'Name': 'jetBlue'},
                      

                                                          'Name': 'San '
                                                                  'Francisco '
                                                                  'International'}},
                                      {   'carrier': {   'Code': 'UA',
                                                         'Name': 'United'},
                                          'duration': 310,
                                          'from_place': {   'Code': 'SFO',
                                                            'Name': 'San '
                                                                    'Francisco '
                                                                    'International'},
                                          'to_place': {   'Code': 'EWR',
                                                          'Name': 'New York '
                                                                  'Newark'}},
           